In [142]:
import tarfile
import pandas as pd
import glob
import os
import numpy as np
from collections import defaultdict

# Load OHSU 

In [2]:
path = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/April2023_gtex_fixed_filters/OHSU_Apr23-2023_fixGTExandExptIDs.tar.gz'

In [3]:
with tarfile.open(path, 'r') as tar:
    names = tar.getnames()
    

In [119]:
# for i in names:
#     print(i)

In [5]:
f1 = 'experiment_map/J_TCGA-A2-A0SX-01A-12R-A084-07_experiment_map.tsv'
f2 = 'expts_per_peptide/J_TCGA-A2-A0SX-01A-12R-A084-07_experiments_per_peptide.tsv'


In [6]:
with tarfile.open(path, 'r') as tar:
    d1 = pd.read_csv(tar.extractfile(f1), sep = '\t')
    d2 = pd.read_csv(tar.extractfile(f2), sep = '\t')

In [7]:
d1.head()

,id,motif_filter,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,normal_cohort_id,#_normal_samples_allowed,reads_per_normal_sample
0,0NN00PC,1,0,0,0,paired,0,0
1,0NNN1PC,1,0,0,0,paired,1,0
2,0NN11PC,1,0,0,0,paired,1,1
3,0NN31PC,1,0,0,0,paired,1,3
4,0NNA1PC,1,0,0,0,paired,1,-1


In [8]:
d2.head()

,peptide_id,peptide_sequence,experiment_ids
0,100001,IESKGNEITLLFMSGIHVSGRGFLASYSVIDKQDLITCLDTASNFL...,0NNN1PC;0NN11PC;0NN31PC;0NNA1PC;0NNN2PC;0NN12P...
1,100002,SDMDLLNPHRKSEVERPASIPLSSGYSTASSDSTPRASTVNYCGLN...,0NNN1PC;0NN11PC;0NN31PC;0NNA1PC;0NNN2PC;0NN12P...
2,100003,SDMDLLNPHRKSEVERPASIPLSSGYSTASSDSTPRASTVNYCGLN...,0NNN1PC;0NN11PC;0NN31PC;0NNA1PC;0NNN2PC;0NN12P...
3,100004,NDIYNFFSPLNPVRVHIEIGPDGRVTGEADVEFATHEDAVAAMSKD...,0NNN1PC;0NN11PC;0NN31PC;0NNA1PC;0NNN2PC;0NN12P...
4,100005,NDIYNFFSPLNPVRVHIEIGPDGRVTGEADVEFATHEDAVAAMSKD...,0NNN1PC;0NN11PC;0NN31PC;0NNA1PC;0NNN2PC;0NN12P...


# Load ETH 

In [9]:
fasta_paths = glob.glob('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_22March_order_wany_wAnnot/*fa')

In [10]:
path_filtered = glob.glob('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_22March_order_wany_wAnnot/*Filt*')

In [11]:
fasta = fasta_paths[0]

In [12]:
fasta

'/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA_Allsamples_all_exp_kmer.fa'

In [13]:
# Note need to match the sample. But in this case not needed
sample = 'TCGA-A2-A0SX-01A-12R-A084-07'

In [14]:
path_filtered_sample = [path for path in path_filtered if sample in path]

In [15]:
len(path_filtered_sample)

65

In [115]:
def path_to_expID(path:str):
    ID = {}
    filter_background = os.path.basename(path).split('_')[-1].replace('.tsv.gz', '')
    filter_foreground = os.path.basename(path).split('_')[-2]
    #print(filter_background, filter_foreground)
    # Extract Values
    ID['filter_background_reads'] = filter_background.split('lim')[-1].split('Across')[0]
    ID['filter_background_samples'] = filter_background.split('Across')[-1]
    ID['filter_background_cohort'] = filter_background.split('Normals')[1].split('Cohort')[0]
    ID['filter_foreground_reads'] = filter_foreground.split('Lim')[-1].split('Across')[0]
    ID['filter_foreground_samples'] = filter_foreground.split('Across')[-1]
    ID['filter_foreground_target'] = (filter_foreground.split('Lim')[1].replace('SampleLim', '').replace('Cohort', ''))
    return ID
    

In [116]:
def expID_to_block(ID_split: dict):
    ID_dict = {'Any': 'A', 'None': 'N', '10': 'X', 'paired': 'P', 'Gtex': 'G', 'Core_GTEx': 'R', 'Full': 'F'}
    motif = 'A' # Any
    for k, v in ID_split.items():
        try:
            assert(v != '10') #replace roman letter
            ID_split[k] = str(int(float(v)))
        except:
            ID_split[k] = ID_dict[v]

    return ID_split['filter_foreground_target'] + \
    ID_split['filter_foreground_reads'] + \
    ID_split['filter_foreground_samples'] + \
    ID_split['filter_background_reads'] + \
    ID_split['filter_background_samples'] + \
    ID_split['filter_background_cohort'] + \
    motif 
    

In [117]:
path_filtered_sample[0]

'/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA-A2-A0SX-01A-12R-A084-07_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlimAnyAcrossAny.tsv.gz'

In [131]:
# Extract the peptides from the fasta
peptides_IDS = []
peptides_sequences = []
kmers = []
with open(fasta, 'r') as f:
    for line in f.readlines():
        if '>' in line:
            kmers.append(line.split(';')[7].split('-')[-1])
            peptides_IDS.append(line.split(';')[0].split('-')[-1])
        else:
            peptides_sequences.append(line.replace('\n',''))




In [182]:
# Extract the occurences of the peptides in the experimental files

kmer_to_filesID = defaultdict(list)

for filtered in path_filtered_sample:
    # GET filtered file ID
    ID_split = path_to_expID(filtered)
    ID_EXPERIMENT = expID_to_block(ID_split)
    # Read filtered file 
    df_filt = pd.read_csv(filtered, sep = '\t')
    for km in set(df_filt['kmer']):
        kmer_to_filesID[km].append(ID_EXPERIMENT)

    
        

In [183]:
# Data wrangling to get the peptide to experiment file 
kmer_to_filesID_ = []
for k, v in kmer_to_filesID.items():
    kmer_to_filesID_.append((k,';'.join(np.unique(v))))

df = pd.DataFrame({'peptide_id': peptides_IDS, \
'peptide_sequence': peptides_sequences, \
'kmers': kmers})

info = pd.DataFrame(kmer_to_filesID_, columns = ['kmers', 'experiment_ids'])

df = df.merge(info, on = 'kmers')

In [ ]:
# GENERATE IDS # Test version #TODO ===> NEED to Generate the parameter file 
collect_IDS = []
for filt_file in path_filtered_sample:
    ID_split = path_to_expID(filt_file)
    ID = expID_to_block(ID_split)
    collect_IDS.append(ID)
    


# TESTS

In [98]:
# TEST ETH 
for pos in np.arange(6):
    res =  np.unique([i[pos] for i in np.unique(collect_IDS)])
    print(f'ID position {pos} unique values {res}')

ID position 0 unique values ['0']
ID position 1 unique values ['0' '2' 'N']
ID position 2 unique values ['1' '5' 'N']
ID position 3 unique values ['0' '1' '3' 'A']
ID position 4 unique values ['1' '2' 'A' 'X']
ID position 5 unique values ['G']


In [44]:
# TEST OHSU 
for pos in np.arange(6):
    res =  np.unique([i[pos] for i in d1['id'].unique()])
    print(f'ID position {pos} unique values {res}')

ID position 0 unique values ['0']
ID position 1 unique values ['2' 'N']
ID position 2 unique values ['1' '5' 'N']
ID position 3 unique values ['0' '1' '3' 'A' 'N']
ID position 4 unique values ['0' '1' '2' 'A' 'X']
ID position 5 unique values ['G' 'P']


In [ ]:
SLACK
# BACKGROUND cohorts we do (cohort_reads, sample_number)- KEEP pipeline as such
# cohort_reads=[0,1,3, Any]
# sample_number=[1,2,10, Any]
# except Any, Any
# FOREGROUND  (cohort_reads, sample_number) means
# sample_reads=[0]
# cohort_reads=[0,2]
# sample_number(rest of cohort) =[1, 5]

In [26]:
print(d1['id'].unique())

['0NN00PC' '0NNN1PC' '0NN11PC' '0NN31PC' '0NNA1PC' '0NNN2PC' '0NN12PC'
 '0NN32PC' '0NNA2PC' '0NNNXPC' '0NN1XPC' '0NN3XPC' '0NNAXPC' '0NNNAPC'
 '0NN1APC' '0NN3APC' '0N100PC' '0N1N1PC' '0N111PC' '0N131PC' '0N1A1PC'
 '0N1N2PC' '0N112PC' '0N132PC' '0N1A2PC' '0N1NXPC' '0N11XPC' '0N13XPC'
 '0N1AXPC' '0N1NAPC' '0N11APC' '0N13APC' '02100PC' '021N1PC' '02111PC'
 '02131PC' '021A1PC' '021N2PC' '02112PC' '02132PC' '021A2PC' '021NXPC'
 '0211XPC' '0213XPC' '021AXPC' '021NAPC' '0211APC' '0213APC' '0N500PC'
 '0N5N1PC' '0N511PC' '0N531PC' '0N5A1PC' '0N5N2PC' '0N512PC' '0N532PC'
 '0N5A2PC' '0N5NXPC' '0N51XPC' '0N53XPC' '0N5AXPC' '0N5NAPC' '0N51APC'
 '0N53APC' '02500PC' '025N1PC' '02511PC' '02531PC' '025A1PC' '025N2PC'
 '02512PC' '02532PC' '025A2PC' '025NXPC' '0251XPC' '0253XPC' '025AXPC'
 '025NAPC' '0251APC' '0253APC' '0NN00PA' '0NNN1PA' '0NN11PA' '0NN31PA'
 '0NNA1PA' '0NNN2PA' '0NN12PA' '0NN32PA' '0NNA2PA' '0NNNXPA' '0NN1XPA'
 '0NN3XPA' '0NNAXPA' '0NNNAPA' '0NN1APA' '0NN3APA' '0N100PA' '0N1N1PA'
 '0N11

In [106]:
for i in np.unique(collect_IDS):
    if i[3] == '0':
        print(i)

00101GA
00102GA
0010AGA
0010XGA
00501GA
00502GA
0050AGA
0050XGA
02101GA
02102GA
0210AGA
0210XGA
02501GA
02502GA
0250AGA
0250XGA
0NN01GA
0NN02GA
0NN0AGA
0NN0XGA


In [108]:
for i in d1['id'].unique():
    if i[3] == '0'  and i[6] == 'A' and i[5] == 'G':
        print(i)

0NN00GA
0N100GA
02100GA
0N500GA
02500GA


In [107]:
for i in d1['id'].unique():
    if i[3] == 'N' and i[6] == 'A' and i[5] == 'G':
        print(i)

0NNN1GA
0NNN2GA
0NNNXGA
0NNNAGA
0N1N1GA
0N1N2GA
0N1NXGA
0N1NAGA
021N1GA
021N2GA
021NXGA
021NAGA
0N5N1GA
0N5N2GA
0N5NXGA
0N5NAGA
025N1GA
025N2GA
025NXGA
025NAGA


In [102]:
d1.loc[d1['id'] == '02100GC']

,id,motif_filter,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,normal_cohort_id,#_normal_samples_allowed,reads_per_normal_sample
192,02100GC,1,0,1,2,GTEx,0,0


In [100]:
d1.loc[d1['id'] == '021N2PA']

,id,motif_filter,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,normal_cohort_id,#_normal_samples_allowed,reads_per_normal_sample
117,021N2PA,0,0,1,2,paired,2,0


In [101]:
d1.loc[d1['id'] == '02102PA']

,id,motif_filter,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,normal_cohort_id,#_normal_samples_allowed,reads_per_normal_sample
